<table width=100%>
  <tr>
    <td>
      <a href="https://colab.research.google.com/github/aurelienmorgan/retrain-pipelines/blob/main/extra/frameworks/Metaflow/metaflow_service.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" /></a>
    </td>
    <td width=405>
      <a href="https://pypi.org/project/retrain-pipelines/" target="_blank"><img src="https://github.com/user-attachments/assets/19725866-13f9-48c1-b958-35c2e014351a" width="150" alt="retrain-pipelines" /></a>
      <a href="https://metaflow.org/" target="_blank"><img src="https://github.com/user-attachments/assets/8085a813-d993-47aa-8992-62123fa39967" width="250" alt="Metaflow" /></a>
    </td>
  </tr>
</table>

<b><center><font size=14em>Stateful Metaflow Service & UI</font></center></b>
<center><em><font size=12em>(2/2) Consume</font></em></center>

<em>The herein notebook is the continuance of the <a href="https://github.com/aurelienmorgan/retrain-pipelines/extra/frameworks/Metaflow/metaflow_service.ipynb" target="_blank">(1/2) start</a> Google Colab notebook</em>

<hr />

In [ ]:
from google.colab import drive
# grant all permission or it fails
drive.mount('/content/drive')

In [ ]:
tunnel_url = None

In [ ]:
# below, must be the same Google Drive location
# as the one set in the "Metaflow Service" notebook
MF_ROOT = "/content/drive/MyDrive/Metaflow_hf"

## Metaflow API

In [ ]:
%%writefile hello_world_flow.py
from metaflow import FlowSpec, step, current, card
from metaflow.cards import Markdown

class HelloWorldFlow(FlowSpec):

    @step
    def start(self):
        print("Hello, World!")
        self.next(self.flow_card)

    @card(id='default')
    @step
    def flow_card(self):
        print("blabla")
        current.card['default'].append(Markdown("blabla"))
        self.next(self.end)

    @step
    def end(self):
        print("Flow Finished")

if __name__ == '__main__':
    HelloWorldFlow()

In [ ]:
datastore_dir = f"{MF_ROOT}/local_datastore/"

# when prompted, enter the URL
# of the tunnel which you established
# in the "Metaflow Service" Colab notebook
import requests
if tunnel_url is not None:
    try:
        response = requests.get(f"{tunnel_url}/service/ping")
        if response.status_code != 200 or response.text != "pong":
            tunnel_url = input("Enter an active tunnel URL:\n")
        else:
            print(tunnel_url)
    except:
        tunnel_url = input("The former endpoint is not reachable. " +
                           "Enter an active tunnel URL:\n")
else:
    tunnel_url = input("Enter an active tunnel URL:\n")

# Launch flow run
! export METAFLOW_SERVICE_URL={tunnel_url}/service/ && \
  export METAFLOW_DEFAULT_METADATA=service && \
  export METAFLOW_DEFAULT_DATASTORE='local' && \
  export METAFLOW_DATASTORE_SYSROOT_LOCAL={datastore_dir} && \
  export USERNAME=user && \
  cd {datastore_dir} && \
  python /content/hello_world_flow.py run

## Metaflow SDK

In [ ]:
!pip install metaflow

In [ ]:
# when prompted, enter the URL
# of the tunnel which you established
# in the "Metaflow Service" Colab notebook

import sys
def unload_package(package_name):
    # Remove the package and its submodules from sys.modules
    names_to_remove = [name for name in sys.modules if name.startswith(package_name)]
    for name in names_to_remove:
        del sys.modules[name]
    if package_name in globals():
        del globals()[package_name]
unload_package('metaflow')

import requests
if tunnel_url is not None:
    try:
        response = requests.get(f"{tunnel_url}/service/ping")
        if response.status_code != 200 or response.text != "pong":
            tunnel_url = input("Enter an active tunnel URL:\n")
        else:
            print(tunnel_url)
    except:
        tunnel_url = input("The former endpoint is not reachable. " +
                           "Enter an active tunnel URL:\n")
else:
    tunnel_url = input("Enter an active tunnel URL:\n")

import os
os.environ['METAFLOW_SERVICE_URL'] = f"{tunnel_url}/service"
os.environ['METAFLOW_DEFAULT_METADATA'] = 'service'
os.environ['USERNAME'] = 'user'

import metaflow

You can use the Metaflow python SDK as usual and, this will work with your Colab-hosted instance&nbsp;:

In [ ]:
list(metaflow.Flow("HelloWorldFlow").runs())[0:10]